In [1]:
import chromadb
import openai
import os

from pymongo import MongoClient
from chromadb.config import Settings
from openai import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langgraph.graph import StateGraph, END
from langgraph.graph.message import MessageGraph
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable, RunnableLambda
from sentence_transformers import SentenceTransformer

In [2]:
#conexões com os bancos de dados

# Conexão com o ChromaClient Persistente
chroma_client = chromadb.PersistentClient(path="../chroma_db")
# Chave da API da openai
openai_api_key = os.getenv("OPENAI_API_KEY")

# Conectar ao MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["rag_db"]
mongo_chunks = mongo_db["chunks"]
mongo_images = mongo_db["images"]



collection = chroma_client.get_collection("rag_embeddings")
image_collection = chroma_client.get_collection(name="image_descriptions")

# Inicializar embeddings
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key) #text-embedding-ada-002

/tmp/ipykernel_19607/1242579972.py:20: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key) #text-embedding-ada-002


In [4]:
#teste para recuperação de contexto de texto
query = "Quais os municípios participantes do APA?"
query_embedding = embedding_model.embed_query(query)
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5
)
print(results)
chunk_ids = [
    result["source_file"] + "_chunk_" + str(result["chunk_index"])
    for result in results["metadatas"][0]
]

chunks_text = []
for chunk_id in chunk_ids:
    chunk = mongo_chunks.find_one({"chunk_id": chunk_id})
    if chunk:
        chunks_text.append(chunk["chunk_text"])
        print("Contexto retirado do documento:")
        print(chunk["pdf_file"])
        print("Link para o documento:")
        print(chunk["source_url"])
print(chunks_text)

{'ids': [['EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_269', 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_267', 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_263', 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_273', 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf_chunk_270']], 'embeddings': None, 'documents': [[None, None, None, None, None]], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'source_file': 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf', 'source_url': 'https://cetesb.sp.gov.br/eiarima/eia/EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf', 'chunk_index': 269}, {'chunk_index': 267, 'source_file': 'EIA-384-24-e-amb-90935-24-Unid-Proc-Oleos-Contam-PCBs-MG-Trafos-Pedreira.pdf', 'source_url

In [5]:
#teste para recuperação de contexto de imagens
def test_image_query(_query):
    try:
        query = _query
        embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
        query_embedding = embedding_model.encode(query).tolist()

        # Realizar a busca no ChromaDB
        results = image_collection.query(
            query_embeddings=[query_embedding],
            n_results=5
        )

        # IDs e distâncias retornadas
        ids = results["ids"][0]
        distances = results["distances"][0]

        # Verificar cada distância
        for id_, distance in zip(ids, distances):
            if distance < 0.65:
                # Buscar o documento no MongoDB
                image_doc = mongo_images.find_one({"id": id_})
                if image_doc:
                    image_url = image_doc.get("url")
                    print(f"Imagem relacionada (distância {distance}): {image_url}")
            else:
                print("não foram encontadas imagens relevantes")

    except Exception as e:
        print(f"Erro ao realizar a busca: {e}")

test_query = "Águas subterranêas em são paulo"
test_image_query(test_query)

Imagem relacionada (distância 0.4930988848209381): https://smastr16.blob.core.windows.net/igeo/2012/03/mapa_aguas_subterraneas.jpg


In [4]:


# Definindo os estados compartilhados
class ChatState(TypedDict):
    input: str
    query: str
    context: str
    image_context: str
    answer: str
    validation: str
    show_context: str

# Agente de busca textual
def retrieve_text_context(state: ChatState) -> ChatState:
    query = state["input"]
    query_embedding = embedding_model.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=5
    )

    chunk_ids = [
        result["source_file"] + "_chunk_" + str(result["chunk_index"])
        for result in results["metadatas"][0]
    ]
    
    show_context = []
    chunks_text = []
    for chunk_id in chunk_ids:
        chunk = mongo_chunks.find_one({"chunk_id": chunk_id})
        if chunk:
            chunks_text.append(chunk["chunk_text"])
            show_context.append(f"Contexto retirado do documento: {chunk['pdf_file']}\nLink para o documento: {chunk['source_url']}")
    return {
        **state,
        "query": query,
        "context": "\n\n".join(chunks_text),
        "show_context": "\n\n".join(show_context)
    }

# Agente de busca de contexto por imagens relacionadas à query
def retrieve_image_context(state: ChatState) -> ChatState:
    query = state["input"]
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
    
    query_embedding = embedding_model.encode(query).tolist()

    # Realizar a busca no ChromaDB
    results = image_collection.query(
        query_embeddings=[query_embedding],
        n_results=5
    )

    # IDs e distâncias retornadas
    ids = results["ids"][0]
    distances = results["distances"][0]
    image_contexts = []
    show_context = state.get("show_context", "")
    for id_, distance in zip(ids, distances):
        if distance < 0.65:
            # Recuperar o ID encontrado
            image_doc = mongo_images.find_one({"id": id_})
            if image_doc:
                image_contexts.append(image_doc["url"])
                show_context += f"\nImagem relacionada (distância {distance}): {image_doc['url']}"
        else:
            show_context += "\nNão foram encontradas imagens relevantes."


    return {
        **state,
        "image_context": "\n\n".join(image_contexts),
        "show_context": show_context.strip()
    }


# Agente que gera a resposta usando OpenAI
def generate_answer(state: ChatState) -> ChatState:
    client = openai.OpenAI(api_key=openai_api_key)

    prompt = f"""
Você é um assistente especialista em licenciamento ambiental, com foco em auxiliar na elaboração de documentos EIA e RIMA. Use exclusivamente os contextos a seguir para responder de forma precisa e técnica.

Contexto textual:
{state['context']}

Contexto de imagens (descrições ou legendas):
{state['image_context']}

Pergunta:
{state['query']}
    """

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente técnico ambiental especializado em EIA e RIMA."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=1000
    )

    return {
        **state,
        "answer": response.choices[0].message.content.strip()
    }

# Agente validador da resposta
def AnswerValidatorAgent(state: ChatState) -> ChatState:
    client = openai.OpenAI(api_key=openai_api_key)

    answer = state["answer"]
    context = state["context"]
    image_context = state["image_context"]
    query = state["query"]

    validation_prompt = f"""
Você é um avaliador técnico. Sua função é atribuir uma **nota de 0 a 10** para a resposta de um assistente, com base nos seguintes critérios:

1. **Relevância** — A resposta está diretamente ligada ao conteúdo do contexto?
2. **Completude** — A resposta aborda todos os aspectos importantes da pergunta?
3. **Precisão** — A resposta está tecnicamente correta?
4. **Consistência** — A resposta está bem estruturada, sem contradições internas?
5. **Conformidade** — A resposta está de acordo com as normas e estilo técnico esperado?

---

**Contexto textual:**
{context}

**Contexto visual:**
{image_context}

**Pergunta:**
{query}

**Resposta do assistente:**
{answer}

---

Dê uma **nota de 0 a 10 para cada critério**, e finalize com uma **breve justificativa** da avaliação geral.

Formato de resposta esperado:
Relevância: X/10  
Completude: X/10  
Precisão: X/10  
Consistência: X/10  
Conformidade: X/10  

Justificativa: [justificativa curta e técnica]
"""

    validation_response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um avaliador técnico de respostas de IA sobre documentos EIA/RIMA."},
            {"role": "user", "content": validation_prompt}
        ],
        temperature=0,
        max_tokens=500
    )

    validation = validation_response.choices[0].message.content.strip()

    return {
        **state,
        "validation": validation
    }


# Construção do grafo
builder = StateGraph(ChatState)

# Adiciona os nós
builder.add_node("text_retriever", retrieve_text_context)
builder.add_node("image_retriever", retrieve_image_context)
builder.add_node("chat", generate_answer)
builder.add_node("validator", AnswerValidatorAgent)

# Define conexões entre os nós
builder.set_entry_point("text_retriever")
builder.add_edge("text_retriever", "image_retriever")
builder.add_edge("image_retriever", "chat")
builder.add_edge("chat", "validator")
builder.add_edge("validator", END)

# Compila o grafo
graph = builder.compile()



In [6]:
#Iteração com o chat
while True:
    user_input = input("Você: ")
    if user_input.lower() in ["sair", "exit", "quit"]:
        break

    result = graph.invoke({"input": user_input})
    print("\nBot:", result["answer"])
    print(result["show_context"])
    print("Validação:\n", result["validation"])


Você:   Durante o processo de estudos para duplicação, foi constatado que é necessário a retirada de vegetação nativa da mata atlântica na extensão da SP-97. Como se deve proceder para diminuir o impacto gerado devido a retirada da mesma segundo a regulação vigente?



Bot: Para minimizar o impacto gerado pela retirada de vegetação nativa da Mata Atlântica na extensão da SP-97, é necessário seguir uma série de procedimentos conforme a regulação vigente. 

1. Solicitação de Anuência Prévia: De acordo com a Lei Estadual nº 10.780, é necessário solicitar uma anuência prévia para a supressão de vegetação primária ou secundária nos estágios médio ou avançado de regeneração no Bioma Mata Atlântica.

2. Planejamento da Supressão: As atividades de supressão devem ser planejadas e executadas de forma a minimizar os impactos sobre as áreas a serem preservadas. Isso inclui ações para possibilitar o afugentamento da fauna para essas áreas e facilitar as operações de resgate de fauna.

3. Compensação Ambiental: A supressão de vegetação natural deve ser compensada por meio da realização de plantios compensatórios, calculados de acordo com a Resolução SMA n°07/2017. Essa resolução dispõe sobre os critérios e parâmetros para compensação ambiental de áreas objeto de

Você:  Para a duplicação da SP-97, a equipe necessita de um mapa que traga informações relativas as bacias hidrográficas do estado de São Paulo. Necessitamos de um mapa de águas subterrânes do estado de são paulo para fazer um estudo de como procederemos quanto a permeabilidade do solo, conseguiria fornecer também informações da regulação vigente relativo a permeabilidade do solo?



Bot: Para a duplicação da SP-97, o Mapa das Águas Subterrâneas do Estado de São Paulo, escala 1:1.000.000 (DAEE et al., 2005) e o Mapa Hidrogeológico da Bacia do rio Juqueri, escala 1:150.000 (Bertolo, 1996) são recursos valiosos. Esses mapas fornecem informações detalhadas sobre as bacias hidrográficas e os recursos hídricos subterrâneos, que são essenciais para a análise da permeabilidade do solo.

Além disso, o estudo "Águas subterrâneas no Estado de São Paulo" publicado pelo DAEE e o Laboratório de Estudo de Bacias (LEBAC) da UNESP em 2013 pode fornecer informações adicionais relevantes.

Quanto à regulação vigente relativa à permeabilidade do solo, a Resolução CONAMA nº 420/2009 estabelece critérios e valores orientadores de qualidade do solo quanto à presença de substâncias químicas e estabelece diretrizes para o gerenciamento ambiental de áreas contaminadas por essas substâncias em decorrência de atividades antrópicas. 

Além disso, a NBR 13292:1995 da ABNT estabelece o método 

Você:  Nossa equipe escreveu um trecho de documentação EIA relativo a replantio de vegetação nativa que será removida dos trechos da via, você poderia verificar se o conteúdo segue as normas ambientais e sugerir melhorias para o foco futuro da equipe? Durante os estudos feitos pela equipe, foi verificado a existência da arvore Vitex polygama Cham. em trechos da estrada. Depois de uma analise cuidadosa da equipe, deseja-se realizar seu replantio no parque natural de chico mendes em Sorocaba, e para cada representante da espécie retirada será plantado um novo dentro dos limites do parque citado.



Bot: A proposta de replantio de Vitex polygama Cham. no parque natural de Chico Mendes em Sorocaba é uma medida compensatória adequada à supressão de vegetação nativa. No entanto, é importante garantir que a espécie seja compatível com o ecossistema do parque e que não cause impactos negativos na biodiversidade local. 

Além disso, é importante considerar a quantidade de indivíduos que serão removidos e garantir que o replantio seja feito em uma proporção adequada. A proporção de 1:1 (um plantado para cada um removido) pode não ser suficiente para compensar a perda de habitat, especialmente se a espécie em questão tiver um papel importante no ecossistema local. 

Sugiro que a equipe considere a adoção de uma proporção maior de replantio, como 3:1 ou até 5:1, dependendo da importância ecológica da espécie e do impacto de sua remoção. 

Além disso, é importante que a equipe inclua no EIA um plano de monitoramento para acompanhar o sucesso do replantio e a adaptação da espécie ao novo am

Você:  sair



Bot: As águas subterrâneas em São Paulo são uma importante fonte de abastecimento para a população e para diversas atividades econômicas. A gestão desses recursos é regulada por várias leis e portarias, como a Portaria DAEE nº 717, de 12 de dezembro de 1996, que estabelece as condições para a implantação de empreendimentos que interfiram com as águas superficiais e subterrâneas.

A Portaria DAEE nº 1.213, de 06 de agosto de 2004, outorga à SABESP a concessão das vazões máximas médias mensais para fins de abastecimento público. Além disso, a recente criação da SP Águas, que substituirá o DAEE, altera as funções das agências reguladoras do estado, incluindo a gestão das outorgas para o uso da água.

A cidade de Pedreira, localizada na APA de Campinas, é um exemplo de local onde a gestão dos recursos hídricos subterrâneos é de suma importância. A extração de água subterrânea é realizada principalmente por meio de poços, que são cuidadosamente monitorados para garantir que a captação não 

Você:  sair
